In [1]:
import numpy as np
import pandas as pd
from pathlib import Path

In [2]:
classes = np.array([str(l) for l in range(28)])
df = pd.read_csv('aug_train.csv'); len(df)
PATH = Path('./')
SAMPLE = Path('sample_submission.csv')
sample_sub = pd.read_csv(PATH/SAMPLE)
test_names = list(sample_sub.Id.values)

In [3]:
def count_classes(df):
    out = np.zeros(28, dtype=int)
    for i in range(len(df)):
        idxs = [int(x) for x in df.Target.iloc[i].split()]
        out[idxs]+=1
    return out

In [4]:
cc = count_classes(df)

In [5]:
hard_classes = np.arange(28)[cc<500]; hard_classes

array([ 8,  9, 10, 15, 17, 20, 27])

# Model ensmeble

In [7]:
model_name = 'ensemble'

In [8]:
r18c = np.load('r18c.npy')

In [9]:
r34 = np.load('predsR34.npy')

In [10]:
r18 = np.load('predsR18.npy')

In [11]:
r18se = np.load('r18se.npy')

In [12]:
r50 = np.load('r50.npy')

In [35]:
ense = (r18+r34+r18se)/3
ense2 = (r18+r34)/2

In [36]:
def preds2frame(preds, th=0.4):
    print(preds.shape)
    res = np.array([" ".join(classes[(np.where(pp>th))])for pp in preds])
    return pd.DataFrame(np.array([test_names, res]).T, columns = ['Id','Predicted'])

In [37]:
df_r18 = preds2frame(r18)
df_r34 = preds2frame(r34)
df_r18c = preds2frame(r18c)
df_ensemble = preds2frame(ense)
df_ensemble2 = preds2frame(ense2)

(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)
(11702, 28)


In [38]:
df = pd.concat([df_r18, 
                df_r34['Predicted'], 
                df_r18c['Predicted'], 
                df_ensemble['Predicted'],
                df_ensemble2['Predicted']],ignore_index=True, axis=1)
df.columns = ['Id', 'r18', 'r34', 'r18c', 'ense', 'ense2']

In [39]:
df.head(15)

,Id,r18,r34,r18c,ense,ense2
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2,2,2,2,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25,5,5 25,5 25,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25,0 5 21 25,0 5 18 21 25,0 5 25,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 25,0 16 18 25,0 16 17 18,0 16 25,0 16 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7,0,7,,7
5,00109f6a-bac8-11e8-b2b7-ac1f6b6435d0,4,4,4,4,4
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 4 23 25,4 5 25,0 4 25,0 4 5 25,0 4 25
7,0018641a-bac9-11e8-b2b8-ac1f6b6435d0,0 25,0 6,0 6 23,0,0
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0,0 26,0 26,0 26,0
9,0026f154-bac6-11e8-b2b7-ac1f6b6435d0,0 2 25,2 4 21 25,2 21 25,2 21 25,2 21 25


In [40]:
idxs = np.where(df['ense'] != df['ense2'])

In [41]:
df.iloc[idxs]

,Id,r18,r34,r18c,ense,ense2
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,7,0,7,,7
6,001765de-bacd-11e8-b2b8-ac1f6b6435d0,0 4 23 25,4 5 25,0 4 25,0 4 5 25,0 4 25
8,00200f22-bad7-11e8-b2b9-ac1f6b6435d0,0,0 26,0 26,0 26,0
10,002729d2-bace-11e8-b2b8-ac1f6b6435d0,25,18,18 25,18 25,25
22,007eca68-bac7-11e8-b2b7-ac1f6b6435d0,0 25,0 16 25,0 25,0 25,0 16 25
25,009131e6-baca-11e8-b2b8-ac1f6b6435d0,0 12,0 12,12,0 12 25,0 12
26,0094159e-bad1-11e8-b2b8-ac1f6b6435d0,,21,21 25,21,
27,0097a5c2-bac9-11e8-b2b8-ac1f6b6435d0,0,0 19,0 17,0 5,0
36,00bb4f2a-bada-11e8-b2b9-ac1f6b6435d0,0 25,25,,,25
38,00bb8fa8-bada-11e8-b2b9-ac1f6b6435d0,0 21 25,25,0 21 25,0 21 25,21 25


In [42]:
df_ensemble.head()

,Id,Predicted
0,00008af0-bad0-11e8-b2b8-ac1f6b6435d0,2
1,0000a892-bacf-11e8-b2b8-ac1f6b6435d0,5 25
2,0006faa6-bac7-11e8-b2b7-ac1f6b6435d0,0 5 25
3,0008baca-bad7-11e8-b2b9-ac1f6b6435d0,0 16 25
4,000cce7e-bad4-11e8-b2b8-ac1f6b6435d0,


In [43]:
df_ensemble.to_csv(f'{model_name}.csv', index=False)

In [44]:
!kaggle competitions submit -c human-protein-atlas-image-classification -f "ensemble.csv" -m "(r18+r34+se18):  model ensemble"

/bin/sh: 1: kaggle: not found
